In [7]:
import requests
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import copy
import time
import datetime
import random

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from futures_sign import send_signed_request, send_public_request, get_timestamp, \
                        BASE_URL, hashing, dispatch_request, get_av_balance
from cred import KEY, SECRET

ModuleNotFoundError: No module named 'futures_sign'

In [3]:
# import talib
# from talib import abstract

In [4]:
# ?talib.ADOSC

In [1]:
# talib.get_function_groups()

In [2]:
# df=get_futures_klines(symbol,limit=500)
# df

In [3]:
# abstract.TRANGE(df)

In [4]:
# DF=PrepareDF(df)
# DF

In [5]:
symbol='ETHUSDT'
client = Client(KEY, SECRET)

maxposition=0.04
stop_percent=0.0009 # 0.01=1%
# eth_proffit_array=[[3,8],[4,2],[8,0],[16,3],[20,1],[25,0],[30,0],[50,0]]
eth_proffit_array=[[4,5],[5,5],[8,0],[10,0],[15,0],[20,0],[25,0],[250,0]]
proffit_array=copy.copy(eth_proffit_array)

pointer=str(random.randint(1000, 9999))


NameError: name 'Client' is not defined

In [6]:
# Get last 500 kandels 5 minutes for Symbol

def get_futures_klines(symbol,limit=500):
    x = requests.get('https://binance.com/fapi/v1/klines?symbol='+symbol+'&limit='+str(limit)+'&interval=5m')
    df=pd.DataFrame(x.json())
    df.columns=['open_time','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    df=df.drop(['d1','d2','d3','d4','d5'],axis=1)
    df['open']=df['open'].astype(float)
    df['high']=df['high'].astype(float)
    df['low']=df['low'].astype(float)
    df['close']=df['close'].astype(float)
    df['volume']=df['volume'].astype(float)
    return(df)

In [9]:
# stop_percent=0.0029
# new = 1226.31
# stop_price=new*(1-stop_percent)
# procent = new / stop_price
# delo = round((new - stop_price), 3) 
# print(delo, procent,stop_price,new, sep = '\n')

In [10]:
# Open position for Sybol with 

def open_position(symbol,s_l,quantity_l):
  
    sprice=get_symbol_price(symbol)
   
    if(s_l=='long'):
        close_price=str(round(sprice*(1+0.0029),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
#                     "price": sprice 

                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/v1/batchOrders', params)
        
        prt(str(responce))
       
       
    if(s_l=='short'):
        close_price=str(round(sprice*(1-0.0029),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
#                     "price": sprice
                }
           ]
        }
        responce = send_signed_request('POST', '/fapi/v1/batchOrders', params)
        prt(str(responce))
        


In [11]:
# Close position for symbol with quantity

def close_position(symbol,s_l,quantity_l):
    
    sprice=get_symbol_price(symbol)
    
    if(s_l=='long'):
        close_price=str(round(sprice*(1-0.0029),2))
        params = {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
               }
        responce = send_signed_request('POST', '/fapi/v1/order', params)
        prt(str(responce))
       

    if(s_l=='short'):
        close_price=str(round(sprice*(1+0.0029),2))
        params = {
                
                    "symbol":symbol,
                    "side": "BUY",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC", 
                    "price": close_price }
        responce = send_signed_request('POST', '/fapi/v1/order', params) 
        prt(str(responce))
        




In [12]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    a = positions[positions['symbol']==symbol]['positionAmt'].astype(float).tolist()[0]
    leverage = int(positions[positions['symbol']==symbol]['leverage'])
    entryprice = positions[positions['symbol']==symbol]['entryPrice']
    profit = float(status['totalUnrealizedProfit'])
    balance = round(float(status['totalWalletBalance']),2)
    if a>0:
        pos = "long"
    elif a<0:
        pos = "short"
    else: 
        pos = ""
    return([pos,a,profit,leverage,balance,round(float(entryprice),3),0])




In [13]:
# Close all orders 

def check_and_close_orders(symbol):
    global isStop 
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

In [14]:
def get_symbol_price(symbol):
    prices = client.get_all_tickers()
    df=pd.DataFrame(prices)
    return float(df[ df['symbol']==symbol]['price'])

In [15]:
# INDICATORS

In [16]:
# To find a slope of price line 
def indSlope(series,n):
    
    array_sl = [j*0 for j in range(n-1)]
    
    for j in range(n,len(series)+1):
        y = series[j-n:j]
        x = np.array(range(n))
        x_sc = (x - x.min())/(x.max() - x.min())
        y_sc = (y - y.min())/(y.max() - y.min())
        x_sc = sm.add_constant(x_sc)
        model = sm.OLS(y_sc,x_sc)
        results = model.fit()
        array_sl.append(results.params[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(array_sl))))
    return np.array(slope_angle)

In [17]:
# True Range and Average True Range indicator

def indATR(source_DF,n):
    df = source_DF.copy()
    df['H-L']=abs(df['high']-df['low'])
    df['H-PC']=abs(df['high']-df['close'].shift(1))
    df['L-PC']=abs(df['low']-df['close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df_temp = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df_temp

In [18]:
# find local mimimum / local maximum

def isLCC(DF,i):
    df=DF.copy()
    LCC=0
    
    if df['close'][i]<=df['close'][i+1] and df['close'][i]<=df['close'][i-1] and df['close'][i+1]>df['close'][i-1]:
        #найдено Дно
        LCC = i-1;
    return LCC

def isHCC(DF,i):
    df=DF.copy()
    HCC=0
    if df['close'][i]>=df['close'][i+1] and df['close'][i]>=df['close'][i-1] and df['close'][i+1]<df['close'][i-1]:
        #найдена вершина
        HCC = i;
    return HCC



In [19]:

def getMaxMinChannel(DF, n):
    maxx=0
    minn=DF['low'].max()
    for i in range (1,n):
        if maxx<DF['high'][len(DF)-i]:
            maxx=DF['high'][len(DF)-i]
        if minn>DF['low'][len(DF)-i]:
            minn=DF['low'][len(DF)-i]
    return(maxx,minn)


In [20]:
# df = get_futures_klines(symbol,limit=500)
# getMaxMinChannel(df,199)

In [21]:
# generate data frame with all needed data

def PrepareDF(DF):
    ohlc = DF.iloc[:,[0,1,2,3,4,5]]
    ohlc.columns = ["date","open","high","low","close","volume"]
    ohlc=ohlc.set_index('date')
    df = indATR(ohlc,14).reset_index()
    df['slope'] = indSlope(df['close'],5)
    df['channel_max'] = df['high'].rolling(10).max()
    df['channel_min'] = df['low'].rolling(10).min()
    df['position_in_channel'] = (df['close']-df['channel_min']) / (df['channel_max']-df['channel_min'])
    df = df.set_index('date')
    df = df.reset_index()
    return(df)

In [22]:
def get_futures_klines_3(symbol,limit=100):
    x = requests.get('https://binance.com/fapi/v1/klines?symbol='+symbol+'&limit='+str(limit)+'&interval=3m')
    df=pd.DataFrame(x.json())
    df.columns=['open_time','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    df=df.drop(['d1','d2','d3','d4','d5'],axis=1)
    df['open']=df['open'].astype(float)
    df['high']=df['high'].astype(float)
    df['low']=df['low'].astype(float)
    df['close']=df['close'].astype(float)
    df['volume']=df['volume'].astype(float)
    return(df)

In [23]:
def check_if_signal(symbol):
    ohlc = get_futures_klines(symbol,100)
    prepared_df = PrepareDF(ohlc)
    signal="" # return value
    
    i=98 # 99 is current kandel which is not closed, 98 is last closed candel, we need 97 to check if it is bottom or top
    
    if isLCC(prepared_df,i-1)>0:
       # found bottom - OPEN LONG
        if prepared_df['position_in_channel'][i-1]<0.5:
            # close to top of channel
            if prepared_df['slope'][i-1]<-45:
                # found a good enter point for LONG
                signal='long'

    if isHCC(prepared_df,i-1)>0:
       # found top - OPEN SHORT
        if prepared_df['position_in_channel'][i-1]>0.5:
            # close to top of channel
            if prepared_df['slope'][i-1]>45:
                # found a good enter point for SHORT
                signal='short'

    return signal
    

In [24]:
telegram_delay=11
bot_token='############################'
chat_id='################'

def getTPSLfrom_telegram():
    strr='https://api.telegram.org/bot'+bot_token+'/getUpdates'
    response = requests.get(strr)
    rs=response.json()
    if(len(rs['result'])>0):
        rs2=rs['result'][-1]
        rs3=rs2['message']
        textt=rs3['text']
        datet=rs3['date']

        if(time.time()-datet)<telegram_delay:
           
            if '0' in textt:
                telegram_bot_sendtext(str(pointer)+': '+"script continue running at "+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
            if 'hello' in textt:
                telegram_bot_sendtext(str(pointer)+': '+'Hello. How are you?')
            if 'close' in textt:
                position=get_opened_positions(symbol)
                open_sl=position[0]
                quantity=position[1]
                #  print(open_sl,quantity)
                close_position(symbol,open_sl,abs(quantity))
            if '1' in textt:
                telegram_bot_sendtext(str(pointer)+': '+"Balance : " +str(round(float(client.futures_account_balance()[6]['balance']),2))+" USD")
            if '2' in textt:
                dqq=get_av_balance(client, time)
                telegram_bot_sendtext(str(dqq)+" $")
            
            if '3' in textt:
                x=get_symbol_price(symbol)
                i= 15
                df=get_futures_klines(symbol,limit=500)
                checpont_max=getMaxMinChannel(df, i)[0]
                checpont_min=getMaxMinChannel(df, i)[1]
                telegram_bot_sendtext(str(pointer)+': '+"Min : " +str(float(checpont_min))+" USD."+" Max :"+str(float(checpont_max))+" USD."+" Now :"+str(float(x))+" USD.")
            if '4' in textt:
                se=get_opened_positions(symbol)
                telegram_bot_sendtext(str(pointer)+': '+"Info : " +str(list(se)))
           
            
def telegram_bot_sendtext(bot_message):
    bot_token2 = bot_token
    bot_chatID = chat_id
    send_text = 'https://api.telegram.org/bot' + bot_token2 + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()

                                      
def prt(message):
    # telegram message
    telegram_bot_sendtext(pointer+': '+message)
    print(pointer+': '+message)


In [25]:
def main(step):
    global proffit_array

    try:
        getTPSLfrom_telegram()
        position=get_opened_positions(symbol)
        open_sl=position[0]
        if open_sl=="": # no position
            signal=check_if_signal(symbol)
            proffit_array=copy.copy(eth_proffit_array)

            if signal=='long':
                open_position(symbol,'long',maxposition)
                
            

            elif signal=='short':
                open_position(symbol,'short',maxposition)
                
        else:

            entry_price=position[5] # enter price
            current_price=get_symbol_price(symbol)
            quantity=position[1]
           
            if open_sl=='long':
                stop_price=entry_price*(1-stop_percent)
                if current_price<stop_price:
                    #stop loss
                   
                    close_position(symbol,'long',abs(quantity))
                    proffit_array=copy.copy(eth_proffit_array)
                else:
                    temp_arr=copy.copy(proffit_array)
                    for j in range(0,len(temp_arr)-1):
                        delta=temp_arr[j][0]
                        contracts=temp_arr[j][1]
                        if(current_price>(entry_price+delta)):
                        # take profit
                            close_position(symbol,'long',abs(round(maxposition*(contracts/10),3)))
                            del proffit_array[0]
                           
            if open_sl=='short':
                stop_price=entry_price*(1+stop_percent)
                if current_price>stop_price:
                    #stop loss
                    close_position(symbol,'short',abs(quantity))
                    proffit_array=copy.copy(eth_proffit_array)
                else:
                    temp_arr=copy.copy(proffit_array)
                    for j in range(0,len(temp_arr)-1):
                        delta=temp_arr[j][0]
                        contracts=temp_arr[j][1]
                        if(current_price<(entry_price-delta)):
                        # take profit
                            close_position(symbol,'short',abs(round(maxposition*(contracts/10),3)))
                            del proffit_array[0]
                                    
    except :
        prt("script continue running at "+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))


In [21]:
starttime=time.time()
timeout = time.time() + 60*60*60  # 60 seconds times 60 meaning the script will run for 12 hr
counterr=1

while time.time() <= timeout:
    try:
        main(counterr)
        counterr=counterr+1
        if counterr>5:
            counterr=1
        time.sleep(20 - ((time.time() - starttime) % 10.0)) # 1 minute interval between each new execution
    except KeyboardInterrupt:
        prt('\n\KeyboardInterrupt. Stopping.')
        exit()
        

ConnectionError: HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot5733462861:AAF_XeEsmIezIEggocJDCcDAFwUh1PcH0KE/sendMessage?chat_id=1094191614&parse_mode=Markdown&text=8949:%20script%20continue%20running%20at%202022-12-09%2023:27:00 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026D7D317730>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

### 